# Imports

In [1]:
import gym
import wandb
import numpy as np
import pandas as pd

In [2]:
from simplenn.activations import LRelu
from simplenn.activations import TanH
from simplenn.activations import Sigmoid
from simplenn.loss.simulation import Simulation
from simplenn.network import Network
from simplenn.network import Layer
from simplenn.optim import Genetic

In [3]:
#env = gym.make('MountainCar-v0')
#env = gym.make('Acrobot-v1')
#env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [4]:
%matplotlib tk

# Performing experiment

In [14]:
# Config dict
config = {}

# Genetic parameters
config["popSize"] = 50
config["nElitism"] = 0
config["generations"] = 20
config["tournamentSize"] = 5
config["mutationProba"] = 0.25
config["mutationScale"] = 0.05
config["mutationRelative"] = False

# Evaluation related parameters
config["nTest"] = 10
config["actionSelectionMode"] = "argmax"
config["verboseFreq"] = 1
config["recordFreq"] = 1

# Wandb only parameters
config["envId"] = env.env.spec.id

In [15]:
def doLearning(config):
    
    Q = Network(
        Simulation(env, config["nTest"], config["actionSelectionMode"], True),
        [
            Layer(env.observation_space.shape[0], 3, Sigmoid(), "kaiming"),
            Layer(3, env.action_space.n, Sigmoid(), "kaiming", True)
        ]
    )
    config["qfunctionClass"] = Q.__class__.__name__
    config["networkTopology"] = Q.printTopology()
    
    algo = Genetic()
    config["algoClass"] = algo.__class__.__name__
    
    wandb.init(project="simple_rl", config=config)
    
    Q = algo.run(
        Q, 
        None, 
        None, 
        config["popSize"], 
        config["tournamentSize"],
        config["nElitism"], 
        config["mutationProba"], 
        config["mutationScale"], 
        config["mutationRelative"], 
        config["generations"], 
        config["verboseFreq"], 
        config["recordFreq"],
        wandb
    )
    
    return (Q, algo)

In [17]:
nRuns = 3

for i in range(nRuns):
    
    print(f"{i+1}/{nRuns}")
    Q, algo = doLearning(config)

1/3


2/3


3/3


# Analyzing the results

### Animation

In [23]:
print(f"Score: {Q.loss.performEpisode(Q, render=True)}")

Score: 355.0


### Score distribution

In [13]:
pd.Series([Q.loss.performEpisode(Q) for _ in range(250)]).describe()

count    250.0
mean     500.0
std        0.0
min      500.0
25%      500.0
50%      500.0
75%      500.0
max      500.0
dtype: float64